In [1]:
import torch

# Local imports
from train import train_model

Using device: cpu


In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda:0


In [3]:
args = {
    "use_small_dataset": False,
    "batch_size": 64,
    "stoppage_epochs": 42,
    "max_epochs": 512,
    "seed": 42,
    "data_path": "../data",
    "frac_train": 0.8,
    "frac_validation": 0.1,
    "frac_test": 0.1,
    "huber_beta": 1.0,
    "weight_decay": 1e-3,
    "lr": 3e-4,
    "scheduler_patience": 10,
    "scheduler_factor": 0.5,
    "hidden_size": 96,
    "num_layers": 7,
    "num_attn_heads": 6,
    "dropout": 0.25,
    "leaky_relu_slope": 0.2,
    "pooling_dim": 96,
}

In [ ]:
import argparse

training_args = argparse.Namespace(**args)
train_model(training_args, device)

Model parameters: 182714


Epoch 1/512: 100%|██████████████████████████████████████████████████████████████████████| 910/910 [02:36<00:00,  5.82it/s]


Epoch 1/512: 
Train Loss = 0.54620, Train MSE = 1.34519, Train MAE = 0.94099, Train Acc = 0.59379
Val Loss = 0.53964, Val MSE = 1.32732, Val MAE = 0.93546, Val Acc = 0.60104


Epoch 2/512: 100%|██████████████████████████████████████████████████████████████████████| 910/910 [00:43<00:00, 20.90it/s]


Epoch 2/512: 
Train Loss = 0.53860, Train MSE = 1.32188, Train MAE = 0.93224, Train Acc = 0.59879
Val Loss = 0.53231, Val MSE = 1.30250, Val MAE = 0.92606, Val Acc = 0.60407


...

Epoch 7/512: 100%|██████████████████████████████████████████████████████████████████████| 910/910 [00:43<00:00, 20.73it/s]


Epoch 7/512: 
Train Loss = 0.49379, Train MSE = 1.19792, Train MAE = 0.87915, Train Acc = 0.63079
Val Loss = 0.48794, Val MSE = 1.19211, Val MAE = 0.87167, Val Acc = 0.64050


Epoch 8/512: 100%|██████████████████████████████████████████████████████████████████████| 910/910 [00:44<00:00, 20.64it/s]


Epoch 8/512: 
Train Loss = 0.48568, Train MSE = 1.17504, Train MAE = 0.86978, Train Acc = 0.63749
Val Loss = 0.48302, Val MSE = 1.18019, Val MAE = 0.86579, Val Acc = 0.64902


Epoch 9/512:  89%|██████████████████████████████████████████████████████████████▎       | 810/910 [00:39<00:04, 20.36it/s]